<a href="https://colab.research.google.com/github/ElijahWandimi/RemoteSensing-Learning/blob/master/GoogleEarthEnginerRemoteSensing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import ee
import folium
from IPython.display import display, Image

In [2]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=z9neCyYA0EvFYi_uPpZqaKfY4vqLf-EnR7l2_iI94Ac&tc=wfNtONWTLKH5DDCnPvHc4hnyLTWg_wNPruEIjERQUt4&cc=eD3maRkpHN35gs5wdHr8TKvMGp-IXRf7x8y_3j_WUJk

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXm0wGsp8p-FjFdRp1GZ2eJXGNyTevU48dYdh-fXeKrRs2BeE70-878

Successfully saved authorization token.


In [3]:
lat = -0.077199
lon = 37.4041
poi = ee.Geometry.Point(lon, lat)
start_date, end_date = '2021-10-01', '2023-10-01'

In [4]:
# get the data
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
          .filterBounds(poi)\
          .filterDate(start_date, end_date)

In [5]:
print(f">>> Total number of images: {landsat.size().getInfo()}")

>>> Total number of images: 45


In [6]:
landsat.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'SR_B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7591, 7741],
   'crs': 'EPSG:32637',
   'crs_transform': [30, 0, 207285, 0, -30, 115815]},
  {'id': 'SR_B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7591, 7741],
   'crs': 'EPSG:32637',
   'crs_transform': [30, 0, 207285, 0, -30, 115815]},
  {'id': 'SR_B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7591, 7741],
   'crs': 'EPSG:32637',
   'crs_transform': [30, 0, 207285, 0, -30, 115815]},
  {'id': 'SR_B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7591, 7741],
   'crs': 'EPSG:32637',
   'crs_transform': [30, 0, 207285, 0, -30, 115815]},
  {'id': 'SR_B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    '

In [7]:
landsat.first().bandNames().getInfo()

['SR_B1',
 'SR_B2',
 'SR_B3',
 'SR_B4',
 'SR_B5',
 'SR_B6',
 'SR_B7',
 'SR_QA_AEROSOL',
 'ST_B10',
 'ST_ATRAN',
 'ST_CDIST',
 'ST_DRAD',
 'ST_EMIS',
 'ST_EMSD',
 'ST_QA',
 'ST_TRAD',
 'ST_URAD',
 'QA_PIXEL',
 'QA_RADSAT']

In [8]:
# image list
landsat_list = landsat.toList(landsat.size());

In [9]:
# image display parameters
img_params = {
    "min": 7000,
    "max": 16000,
    "dimensions": 800, # square size in pixels
    "bands": ['SR_B4', 'SR_B3', 'SR_B2'] # display bands (r, g, b)
}

In [10]:
data = []

for i in range(landsat.size().getInfo()):
  date_captured = ee.Image(landsat_list.get(i)).get('DATA_ACQUIRED').getInfo()
  cloud_cover = ee.Image(landsat_list.get(i)).get('CLOUD_COVER').getInfo()

  print(f"Image >> {i, date_captured} cloud cover: {cloud_cover}")

  this_data = [i, date_captured, cloud_cover]
  data.append(this_data)

df = pd.DataFrame(data, columns = ['imageId', 'dataCaptured', 'cloudCover'])

display(Image(url = ee.Image(landsat_list.get(24)).getThumbURL(img_params)))


Image >> (0, None) cloud cover: 20.08
Image >> (1, None) cloud cover: 75.08
Image >> (2, None) cloud cover: 19.23
Image >> (3, None) cloud cover: 24.51
Image >> (4, None) cloud cover: 27.67
Image >> (5, None) cloud cover: 34.2
Image >> (6, None) cloud cover: 40.66
Image >> (7, None) cloud cover: 20.22
Image >> (8, None) cloud cover: 1.5
Image >> (9, None) cloud cover: 49.73
Image >> (10, None) cloud cover: 0.15
Image >> (11, None) cloud cover: 45.09
Image >> (12, None) cloud cover: 37.6
Image >> (13, None) cloud cover: 12.22
Image >> (14, None) cloud cover: 22.58
Image >> (15, None) cloud cover: 22.52
Image >> (16, None) cloud cover: 20.07
Image >> (17, None) cloud cover: 26.27
Image >> (18, None) cloud cover: 59.58
Image >> (19, None) cloud cover: 55.34
Image >> (20, None) cloud cover: 9.16
Image >> (21, None) cloud cover: 79.95
Image >> (22, None) cloud cover: 24.16
Image >> (23, None) cloud cover: 37.44
Image >> (24, None) cloud cover: 24.27
Image >> (25, None) cloud cover: 31.19
Im

In [11]:
# define region of interest
roi = poi.buffer(20000)
img_params['region'] = roi

In [12]:
rows, cols = 5, 3
for i in range(min(rows*cols, 15)):
  date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()
  cloud = ee.Image(landsat_list.get(i)).get('CLOUD_COVER').getInfo()

  print(f"{i}: {date}, {cloud}")
  display(Image(url = ee.Image(landsat_list.get(i)).getThumbUrl(img_params)))

0: 2021-10-04, 20.08


1: 2021-10-20, 75.08


2: 2021-11-05, 19.23


3: 2021-11-21, 24.51


4: 2021-12-07, 27.67


...

10: 2022-03-13, 0.15


11: 2022-03-29, 45.09


12: 2022-04-14, 37.6


13: 2022-04-30, 12.22


14: 2022-05-16, 22.58


In [13]:
"""
NDVI = (NIR - RED) / (NIR + RED)
NIR = Near Infrared (B5)
"""
palette = ['red', 'yellow', 'green']

ndvi_params = {
    'min': 0,
    'max': 0.4,
    'dimensions': 512,
    'palette': palette,
    'region': roi
}

In [14]:
rows, cols = 3, 3
for i in range(min(rows*cols, 9)):
  date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()
  cloud = ee.Image(landsat_list.get(i)).get('CLOUD_COVER').getInfo()

  print(f"{i}: {date}, {cloud}")
  display(Image(url = ee.Image(landsat_list.get(i)).normalizedDifference(['SR_B5', 'SR_B4']).getThumbUrl(ndvi_params)))

0: 2021-10-04, 20.08


1: 2021-10-20, 75.08


2: 2021-11-05, 19.23


3: 2021-11-21, 24.51


4: 2021-12-07, 27.67


5: 2021-12-23, 34.2


6: 2022-01-08, 40.66


7: 2022-01-24, 20.22


8: 2022-02-09, 1.5


In [15]:
folium.Map(location = (lat, lon))

In [ ]:
mt_map = folium.Map(location = [lat, lon], zoom_start = 10)

for i in range(10, 17):
  date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()
  foliumgee.add_ee_layer(mt_map, ee.Image(landsat_list.get(i)).normalizedDifference(['SR_B5', 'SR_B4']), ndvi_params, name = date)

folium.LayerControl(collapsed = False).add_to(mt_map)

display(mt_map)